In [29]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

In [1]:
import json
import string
import re
corpus = []
# read json file
fileName = 'Electronics_5.json'
# read
with open(fileName, "r") as read_file:
    i = 0
    l = read_file.readline()
    while l and i < 1004:
        i+=1
        l_dict = json.loads(l)
        review = l_dict["reviewText"]
        r_list = review.split('.')
        for r in r_list:
            r = re.sub('[^a-zA-Z]', ' ', r)
            r = r.lower()
            r = r.split()
            r = [w for w in r if not w in set(string.punctuation)]
            r = ' '.join(r)
            if r != '':
                corpus.append(r)
        l = read_file.readline()

In [2]:
print(corpus[:10])

['we got this gps for my husband who is an otr over the road trucker', 'very impressed with the shipping time it arrived a few days earlier than expected', 'within a week of use however it started freezing up', 'could of just been a glitch in that unit', 'worked great when it worked will work great for the normal person as well but does have the trucker option', 'the big truck routes tells you when a scale is coming up ect', 'love the bigger screen the ease of use the ease of putting addresses into memory', 'nothing really bad to say about the unit with the exception of it freezing which is probably one in a million and that s just my luck', 'i contacted the seller and within minutes of my email i received a email back with instructions for an exchange very impressed all the way around', 'i m a professional otr truck driver and i bought a tnd at a truck stop hoping to make my life easier']


In [3]:
from nltk.corpus import stopwords
def remove_stop_words(corpus):
    stop_words = set(stopwords.words('english'))
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [4]:
corpus = remove_stop_words(corpus)

In [5]:
words = []
for text in corpus:
    for word in text.split(' '):
        #print(word)
        words.append(word)

print(len(words))
words = set(words)

86018


here we have word set by which we will have word vector

In [6]:
print(words)

{'', 'foolishly', 'slipped', 'limits', 'este', 'cartridge', 'cited', 'uncomfortable', 'javoedge', 'thick', 'shoulders', 'outstanding', 'singable', 'upon', 'bringing', 'aside', 'assuming', 'funds', 'decrypted', 'gadget', 'depends', 'pdfs', 'proce', 'sister', 'caring', 'plain', 'expected', 'problems', 'artist', 'nitpicky', 'tchotchkes', 'caved', 'everyone', 'areas', 'julie', 'poi', 'gotten', 'popups', 'mouth', 'carrying', 'downsides', 'impression', 'credits', 'cursor', 'pick', 'cinemanow', 'pulling', 'calendar', 'or', 'straw', 'addressesweb', 'debit', 'protective', 'emagazines', 'encoding', 'adb', 'comcast', 'peculiar', 'locked', 'setback', 'manly', 'walled', 'tape', 'filters', 'recall', 'powering', 'straightaway', 'machines', 'superfluous', 'mail', 'public', 'toys', 'usb', 'rate', 'tread', 'microphones', 'eap', 'urge', 'geared', 'doo', 'joke', 'ans', 'force', 'audio', 'youll', 'adjusting', 'microcenter', 'ipod', 'anyways', 'download', 'reappear', 'bodes', 'earbuds', 'blown', 'struggles'

In [36]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []
for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [37]:
import pandas as pd
for text in corpus:
    print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])

got gps husband otr road trucker
impressed shipping time arrived days earlier expected
within week use however started freezing
could glitch unit
worked great worked work great normal person well the trucker option
big truck routes tells scale coming ect
love bigger screen the ease use the ease of putting addresses memory
nothing really bad say unit the exception freezing probably one million luck
contacted seller within minutes email i received email back instructions exchange impressed the way around
got gps husband otr road trucker
impressed shipping time arrived days earlier expected
within week use however started freezing
could glitch unit
worked great worked work great normal person well the trucker option
big truck routes tells scale coming ect
love bigger screen the ease use the ease of putting addresses memory
nothing really bad say unit the exception freezing probably one million luck
contacted seller within minutes email i received email back instructions exchange impressed

In [38]:
df.head(10)

,input,label
0,got,gps
1,got,husband
2,gps,got
3,gps,husband
4,gps,otr
5,husband,got
6,husband,gps
7,husband,otr
8,husband,road
9,otr,gps


In [39]:
df.shape

(260000, 2)

In [40]:
len(word2int)

65

In [41]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np

ONE_HOT_DIM = len(words)


def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] 
Y = [] 
for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))


X_train = np.asarray(X)
Y_train = np.asarray(Y)


x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))


EMBEDDING_DIM = 50


W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)


W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))


loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))


train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [42]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 20000
for i in range(iteration):

    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 3000 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

iteration 0 loss is :  16.768484


KeyboardInterrupt: 

In [ ]:

vectors = sess.run(W1 + b1)
print(vectors)

[[ 0.939799   -0.07269198 -0.97047895 ...  0.7051742  -0.15360045
   0.51146656]
 [-0.1873407  -1.1416229   0.5282674  ...  0.16597298  0.7074446
  -0.72289747]
 [ 1.52764     0.25991395 -0.7446072  ...  1.1543721   1.399253
  -0.36812222]
 ...
 [-0.43489447 -0.7359016  -0.1985785  ...  0.23103496 -0.19230562
   0.49056885]
 [ 1.794864    2.3677323  -0.14232966 ... -0.20893413  0.12733576
   1.1365646 ]
 [-1.3320976  -1.1418248  -1.0579894  ...  0.19501397 -0.72448903
   0.29870006]]


In [ ]:
w2v_df = pd.DataFrame(vectors, columns = ['x'+str(i) for i in range(1, EMBEDDING_DIM+1)])
w2v_df['word'] = words
w2v_df = w2v_df[['word'] + ['x'+str(i) for i in range(1, EMBEDDING_DIM+1)]]
w2v_df

,word,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50
0,days,0.939799,-0.072692,-0.970479,0.764277,0.005084,0.117511,-0.822014,0.656016,0.222889,...,0.246136,-0.994582,1.446947,-0.132918,1.080357,1.574163,1.063431,0.705174,-0.153600,0.511467
1,glitch,-0.187341,-1.141623,0.528267,0.468640,-1.393529,0.200473,0.410444,0.088482,-1.088427,...,-0.402051,1.283518,-0.739742,0.616822,0.633267,1.597046,1.230733,0.165973,0.707445,-0.722897
2,unit,1.527640,0.259914,-0.744607,-1.541466,1.504610,0.358284,-0.055283,0.531721,0.011619,...,-0.790788,-0.022923,0.614775,0.558936,0.518648,-0.755605,0.235046,1.154372,1.399253,-0.368122
3,love,1.088039,0.799649,-0.056783,-0.642312,0.689861,0.118699,-0.260751,1.538907,-0.531299,...,-0.385175,1.642984,-0.324092,1.695246,0.725493,-0.499327,0.758729,0.740417,-0.460985,1.670823
4,exchange,0.041626,0.385806,-0.161574,1.478175,-0.883460,0.212744,-0.285856,0.071793,1.351103,...,-2.041175,1.338348,0.501510,-1.734342,0.717494,-0.605008,0.322809,0.563093,-0.080149,-0.035231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,of,0.869975,-0.539592,1.452256,1.815241,0.302172,1.918517,-1.295815,-0.315433,0.311235,...,-0.060091,-1.204293,-0.887394,0.324741,0.756478,-0.259807,-0.073089,-0.940413,0.499771,0.622067
61,impressed,0.326595,-0.304986,2.031691,-0.350441,-1.221668,0.026647,0.948178,0.335767,0.458206,...,-0.501210,-0.162042,-0.170697,-1.744612,0.302444,1.170005,0.598283,-2.103957,0.710737,-0.583350
62,coming,-0.434894,-0.735902,-0.198579,0.900158,1.309612,0.546925,1.014227,0.202772,0.718133,...,0.642574,-0.287559,-0.582734,-0.065121,0.664541,-1.016463,0.321847,0.231035,-0.192306,0.490569
63,well,1.794864,2.367732,-0.142330,0.735045,0.674124,-0.099631,0.088953,0.732747,1.582632,...,-0.045020,-0.176293,-0.606731,-1.478055,0.294860,-0.703321,0.895588,-0.208934,0.127336,1.136565


In [ ]:
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots()

# for word, x1, x2 in zip(w2v_df['word'], w2v_df['x1'], w2v_df['x2']):
#     ax.annotate(word, (x1,x2 ))
    
# PADDING = 1.0
# x_axis_min = np.amin(vectors, axis=0)[0] - PADDING
# y_axis_min = np.amin(vectors, axis=0)[1] - PADDING
# x_axis_max = np.amax(vectors, axis=0)[0] + PADDING
# y_axis_max = np.amax(vectors, axis=0)[1] + PADDING
 
# plt.xlim(x_axis_min,x_axis_max)
# plt.ylim(y_axis_min,y_axis_max)
# plt.rcParams["figure.figsize"] = (10,10)

# plt.show()

In [ ]:
# save model to csv
w2v_df.to_csv('w2v_model.csv', index=False)

In [ ]:
def cosine_similarity(x1, x2):
    return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

In [ ]:
def findSimilarity(word, model, n = 10):
    # get the index of the word
  
    word_vec = list(w2v_df[w2v_df['word'] == word].values[0])[1:]
    similarities = []
    for index, row in w2v_df.iterrows():
        if row['word'] == word:
            continue
        similarities.append((row['word'], cosine_similarity(word_vec, list(row)[1:])))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:n]
    

In [ ]:
print(findSimilarity('bigger', w2v_df, n=10))

[('well', 0.33368500147873625), ('got', 0.31847343848889964), ('ease', 0.2768834139469381), ('love', 0.22028789144405328), ('coming', 0.21684597084694407), ('option', 0.20082822510856896), ('contacted', 0.19742697251107186), ('person', 0.18055928692072787), ('instructions', 0.1761952300034511), ('nothing', 0.17218533851667098)]


In [ ]:
# find the row where word column is 'love'


[1.0880392789840698,
 0.799649178981781,
 -0.056783244013786316,
 -0.6423118710517883,
 0.6898607015609741,
 0.11869940906763077,
 -0.2607506513595581,
 1.5389068126678467,
 -0.5312985777854919,
 -1.1486507654190063,
 -2.397254228591919,
 -0.12846669554710388,
 1.529862642288208,
 2.1785366535186768,
 0.4197899401187897,
 0.725969135761261,
 1.2426855564117432,
 -2.1740548610687256,
 1.2013800144195557,
 1.6401646137237549,
 -0.08999241143465042,
 -0.8410106897354126,
 0.7160742282867432,
 0.3351263999938965,
 0.13558551669120789,
 1.3662101030349731,
 1.1011170148849487,
 0.4691050052642822,
 0.7881497144699097,
 0.5187317132949829,
 0.7334776520729065,
 0.18310797214508057,
 -1.2190450429916382,
 -0.6486284136772156,
 0.8288964629173279,
 0.4706714451313019,
 1.9058754444122314,
 0.18385061621665955,
 -0.659069299697876,
 -0.0752599835395813,
 -0.38517534732818604,
 1.6429835557937622,
 -0.32409191131591797,
 1.6952455043792725,
 0.7254928946495056,
 -0.4993272125720978,
 0.758729398